In [ ]:
# Import necessary modules and libraries
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
import numpy as np


In [ ]:
# Define the problem size (number of positions) and the number of sets
PROBLEM_SIZE = 50
NUM_SETS = 100

# Generate a collection of binary sets, each containing random True/False values
SETS = tuple(
    np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)

# Define a named tuple 'State' to represent the current state with sets taken and not taken
State = namedtuple('State', ['taken', 'not_taken'])

In [ ]:

# Define a function 'goal_check' to check if the current state meets the goal
def goal_check(state):
    return np.all(reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ))

# Define a heuristic function to estimate the remaining distance to the goal
def heuristic(state):
    return PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        ))

# Define a cost function to represent the cost of reaching a state (number of steps taken)
def cost(state):
    return len(state.taken)

In [ ]:
# Initialize a priority queue 'frontier' to manage state exploration
frontier = PriorityQueue()

# Initialize the initial state with no sets taken and all sets not taken
initial_state = State(set(), set(range(NUM_SETS)))

# Put the initial state into the frontier with a priority based on the heuristic
frontier.put((heuristic(initial_state), 0, initial_state))

# Initialize a set to keep track of explored states
explored = set()

# Initialize a counter to keep track of the number of steps taken
counter = 0

# Perform the A* search loop
while not frontier.empty():
    # Get the state with the lowest priority (based on heuristic and cost)
    _, steps, current_state = frontier.get()
    counter += 1

    # Check if the current state satisfies the goal condition
    if goal_check(current_state):
        print(
            f"Solved in {steps:,} steps ({len(current_state.taken)} tiles)"
        )
        break

    # Print information about the current state and cost
    print(f"Step {counter}:")
    print(f"Current State - Taken: {current_state.taken}, Not Taken: {current_state.not_taken}")
    print(f"Cost: {steps}")

    # Add the current state to the set of explored states
    explored.add(frozenset(current_state.taken))

    # Generate new states by selecting/deselecting sets
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )

        # Check if the new state has not been explored
        if frozenset(new_state.taken) not in explored:
            new_cost = steps + 1
            priority = new_cost + heuristic(new_state)
            frontier.put((priority, new_cost, new_state))

    # Print a newline to separate steps
    print("\n")

# Print a message when the goal state is reached
print("Goal state reached.")